In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [38]:
from tardis import constants as cons
from tardis.montecarlo.packet_source import BasePacketSource
from tardis import run_tardis
from tardis.io.config_reader import Configuration
from tardis.io.atom_data import AtomData

In [39]:
config = Configuration.from_yaml("/Users/anirbandutta/Downloads/tardis/docs/tardis_example.yml")

config.atom_data = '/Users/anirbandutta/Downloads/kurucz_cd23_chianti_H_He.h5'

In [40]:
from astropy import units as u

In [41]:
class truncblackbodysource(BasePacketSource):

    def __init__(self, base_seed, truncation_wavelength):
        super().__init__(base_seed)
        self.truncation_wavelength = truncation_wavelength

    def create_packets(self, T, no_of_packets, radius, drawing_sample_size=None):

        radii = np.ones(no_of_packets) * radius
        mus = self.create_zero_limb_darkening_packet_mus(no_of_packets)
        energies = self.create_uniform_packet_energies(no_of_packets)

        if drawing_sample_size is None:
            drawing_sample_size = 2 * no_of_packets

        truncation_frequency = u.Quantity(self.truncation_wavelength, u.Angstrom).to(u.Hz, equivalencies=u.spectral()).value


        nus = self.create_blackbody_packet_nus(T, drawing_sample_size)
        nus = nus[nus < truncation_frequency][:no_of_packets]

        while nus.shape[0] < no_of_packets:
            additional_nus = self.create_blackbody_packet_nus(T, drawing_sample_size)
            mask = additional_nus < truncation_frequency
            additional_nus = additional_nus[mask][:no_of_packets]
            nus = np.hstack([nus, additional_nus])[:no_of_packets]

        return radii, nus, mus, energies


In [24]:
packet_source = truncblackbodysource(base_seed=2537, truncation_wavelength=2000)

In [34]:
model_customised = run_tardis('/Users/anirbandutta/Downloads/tardis/docs/tardis_example.yml',
                              packet_source=packet_source, show_convergence_plots=False, 
                              atom_data=atom_data)
# model_norm = run_tardis("/Users/anirbandutta/Downloads/tardis/docs/tardis_example.yml", show_convergence_plots=False)

TypeError: concatenate() got an unexpected keyword argument 'dtype'

In [31]:
atom_data = AtomData.from_hdf(config.atom_data)

[tardis.io.atom_data.base][INFO   ]  
	Reading Atom Data with: UUID = 6f7b09e887a311e7a06b246e96350010 MD5  = 864f1753714343c41f99cb065710cace  (base.py:249)
[tardis.io.atom_data.base][INFO   ]  
	Non provided Atomic Data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:253)


In [32]:
atom_data

<Atomic Data UUID=6f7b09e887a311e7a06b246e96350010 MD5=864f1753714343c41f99cb065710cace Lines=271741 Levels=24806>

In [35]:
packet_source

In [43]:
import tardis
tardis.__version__

'23.5.24.0.dev4374+ga9bde5c'